# Linear Regression for Traffic Density

In [1]:
import pandas as pd
import sys

In [2]:
# #importing the traffic labels csv that was initially calculated using equal weightage for the weights
# traffic_labels = pd.read_csv('traffic_density_labels.csv')
# traffic_labels

In [4]:
#importing the checkpoint5
checkpoint5 = pd.read_csv('../kaggle_test_traffic.csv')
checkpoint5

,key,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,day,weekend,holiday,peak_hour,hotspot,dist,traffic_density
0,2015-01-27 13:08:24.0000002,-73.973320,40.763805,-73.981430,40.743835,1,1,0,0,0,0,2.323260,0.083333
1,2015-01-27 13:08:24.0000003,-73.986862,40.719383,-73.998886,40.739201,1,1,0,0,0,0,2.425353,0.083333
2,2011-10-08 11:53:44.0000002,-73.982524,40.751260,-73.979654,40.746139,1,5,1,0,0,0,0.618628,0.400000
3,2012-12-01 21:12:12.0000002,-73.981160,40.767807,-73.990448,40.751635,1,5,1,0,0,0,1.961033,0.400000
4,2012-12-01 21:12:12.0000003,-73.966046,40.789775,-73.988565,40.744427,1,5,1,0,0,1,5.387301,0.700000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9909,2015-05-10 12:37:51.0000002,-73.968124,40.796997,-73.955643,40.780388,6,6,1,0,0,0,2.124874,0.400000
9910,2015-01-12 17:05:51.0000001,-73.945511,40.803600,-73.960213,40.776371,6,0,0,0,1,0,3.270969,0.383333
9911,2015-04-19 20:44:15.0000001,-73.991600,40.726608,-73.789742,40.647011,6,6,1,0,0,0,19.183941,0.400000
9912,2015-01-31 01:05:19.0000005,-73.985573,40.735432,-73.939178,40.801731,6,5,1,0,0,0,8.343486,0.400000


In [5]:
traffic_labels = checkpoint5[['traffic_density']]
traffic_labels

,traffic_density
0,0.083333
1,0.083333
2,0.400000
3,0.400000
4,0.700000
...,...
9909,0.400000
9910,0.383333
9911,0.400000
9912,0.400000


In [6]:
#Getting the summary of the dataset
shape = checkpoint5.shape

NUM_ROWS = shape[0]
NUM_COLS = shape[1]

print("Number of Rows:", NUM_ROWS)
print("NUmber of Columns:", NUM_COLS)

Number of Rows: 9914
NUmber of Columns: 13


In [7]:
#converting the weekend column from boolean to binary
checkpoint5.replace({False: 0, True: 1}, inplace=True)
checkpoint5

,key,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,day,weekend,holiday,peak_hour,hotspot,dist,traffic_density
0,2015-01-27 13:08:24.0000002,-73.973320,40.763805,-73.981430,40.743835,1,1,0,0,0,0,2.323260,0.083333
1,2015-01-27 13:08:24.0000003,-73.986862,40.719383,-73.998886,40.739201,1,1,0,0,0,0,2.425353,0.083333
2,2011-10-08 11:53:44.0000002,-73.982524,40.751260,-73.979654,40.746139,1,5,1,0,0,0,0.618628,0.400000
3,2012-12-01 21:12:12.0000002,-73.981160,40.767807,-73.990448,40.751635,1,5,1,0,0,0,1.961033,0.400000
4,2012-12-01 21:12:12.0000003,-73.966046,40.789775,-73.988565,40.744427,1,5,1,0,0,1,5.387301,0.700000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9909,2015-05-10 12:37:51.0000002,-73.968124,40.796997,-73.955643,40.780388,6,6,1,0,0,0,2.124874,0.400000
9910,2015-01-12 17:05:51.0000001,-73.945511,40.803600,-73.960213,40.776371,6,0,0,0,1,0,3.270969,0.383333
9911,2015-04-19 20:44:15.0000001,-73.991600,40.726608,-73.789742,40.647011,6,6,1,0,0,0,19.183941,0.400000
9912,2015-01-31 01:05:19.0000005,-73.985573,40.735432,-73.939178,40.801731,6,5,1,0,0,0,8.343486,0.400000


 ### Traffic = A.max(weekend V ph, 0.05 ) + B.max(Time_peak_hr,0.1) + C.max(hotspot, 0.1)  + e(small constant)

#### Adding Epsilon into the Dataframe

In [8]:
#initialising EPSILON
EPSILON = sys.float_info.epsilon
EPSILON

2.220446049250313e-16

In [9]:
EPSILON_LIST = []
for i in range(NUM_ROWS):
    EPSILON_LIST.append(EPSILON)

print("Length:", len(EPSILON_LIST))

Length: 9914


In [10]:
#convert it to df
own_eqn_df = pd.DataFrame(EPSILON_LIST)

In [11]:
#Renaming the header
own_eqn_df.rename(columns={0:"e"}, inplace=True)
own_eqn_df

,e
0,2.220446e-16
1,2.220446e-16
2,2.220446e-16
3,2.220446e-16
4,2.220446e-16
...,...
9909,2.220446e-16
9910,2.220446e-16
9911,2.220446e-16
9912,2.220446e-16


#### Defining Functions for the terms of the Formula

In [12]:
#First term of the formula
def create_day_feature(weekend, holiday):
    day = max((weekend or holiday), 0.05)
    return day

#Second term of the formula
def create_time_feature(peak_hr):
    time = max(peak_hr, 0.1)
    return time

#Third term of the formula
def create_location_feature(hotspot):
    location = max(hotspot, 0.1)
    return location

#### Creating the Various Columns

In [13]:
#Iterating through the dataset
day_list = []
time_list = []
location_list = []

for index, row in checkpoint5.iterrows():
    day_value = create_day_feature(row['weekend'], row['holiday'])
    time_value = create_time_feature(row['peak_hour'])
    location_value = create_location_feature(row['hotspot'])
    
    day_list.insert(index,day_value)
    time_list.insert(index,time_value)
    location_list.insert(index,location_value)

#check 
if len(day_list) == NUM_ROWS:
    print('Successful Creation of day_list!')
if len(time_list) == NUM_ROWS:
    print('Successful Creation of time_list!')
if len(location_list) == NUM_ROWS:
    print('Successful Creation of location_list!')
else:
    print("Failed!")

Successful Creation of day_list!
Successful Creation of time_list!
Successful Creation of location_list!


In [14]:
#Converting the lists into dataframes
day_df = pd.DataFrame(day_list)
time_df = pd.DataFrame(time_list)
location_df = pd.DataFrame(location_list)

In [15]:
day_df

,0
0,0.05
1,0.05
2,1.00
3,1.00
4,1.00
...,...
9909,1.00
9910,0.05
9911,1.00
9912,1.00


In [16]:
#Joining the Dataframes
# own_eqn_df.join(traffic_labels)

In [17]:
own_eqn_df['day'] = day_df
own_eqn_df['time'] = time_df
own_eqn_df['location'] = location_df
# own_eqn_df.join(location_df)

In [18]:
own_eqn_df['traffic_density'] = traffic_labels

In [19]:
own_eqn_df

,e,day,time,location,traffic_density
0,2.220446e-16,0.05,0.1,0.1,0.083333
1,2.220446e-16,0.05,0.1,0.1,0.083333
2,2.220446e-16,1.00,0.1,0.1,0.400000
3,2.220446e-16,1.00,0.1,0.1,0.400000
4,2.220446e-16,1.00,0.1,1.0,0.700000
...,...,...,...,...,...
9909,2.220446e-16,1.00,0.1,0.1,0.400000
9910,2.220446e-16,0.05,1.0,0.1,0.383333
9911,2.220446e-16,1.00,0.1,0.1,0.400000
9912,2.220446e-16,1.00,0.1,0.1,0.400000


In [20]:
own_eqn_df.to_csv('test_subdataset.csv',index=False)

In [21]:
own_eqn_df

,e,day,time,location,traffic_density
0,2.220446e-16,0.05,0.1,0.1,0.083333
1,2.220446e-16,0.05,0.1,0.1,0.083333
2,2.220446e-16,1.00,0.1,0.1,0.400000
3,2.220446e-16,1.00,0.1,0.1,0.400000
4,2.220446e-16,1.00,0.1,1.0,0.700000
...,...,...,...,...,...
9909,2.220446e-16,1.00,0.1,0.1,0.400000
9910,2.220446e-16,0.05,1.0,0.1,0.383333
9911,2.220446e-16,1.00,0.1,0.1,0.400000
9912,2.220446e-16,1.00,0.1,0.1,0.400000
